In [1]:
"""
安装pySWATPlus
   SPOTPY
   mpi4py
   MS mpi

0、pySWATPlus 下载rar解压，用anaconda prompt界面cd到解压的文件夹，用pytohn setup.py install安装

一、TxtinoutReader
1.必须指定工作路径，否则工作路径会变成copy里的路径
2.必须使用绝对路径
3.网页教程的参数格式不对
4.改变参数，不指定行id时，用 None

二、FileReader
1.读取文件不指定index时，必须用None,不可省略
2.usecols省略的话，读取的df为空

三、定义SWAT函数给SPOTPY用时注意以下几点
1、SWAT模型的参数定义不要超过上下界，如(0, 1)用(0.001, 0.999)代替
2、用pySWATPlus定义函数给simulation用时，尽量把只修改一次文件的语句放函数外面
   例如， reader.set_beginning_and_end_year(2017, 2018)
         reader.set_warmup(1)
         reader.enable_object_in_print_prt("channel_sd", True, False, False, False)
   这些直接在root文件夹里手动改就行，不要写成代码，因为用mpiexec并行处理时，会导致文件被意想不到的修改
"""
%reset -f
import os
import pandas as pd
import numpy as np
import spotpy as sp
import shutil
import mpi4py
import sys
from pySWATPlus.TxtinoutReader import TxtinoutReader
from pySWATPlus.FileReader import FileReader
from matplotlib import pyplot as plt
from datetime import datetime
cwd = "E:/SPOTPY-and-pySWATPlus"

# 用pySWATPlus定义huron_swat函数

In [2]:
def huron_swat(reader, params, copy_path, show_output=False, delete_copy=True):
    result = reader.copy_and_run(dir=copy_path,
                                 params=params,
                                 show_output=show_output
                                 )
    reader = FileReader(os.path.join(result, "channel_sd_day.txt"),
                        has_units = True,
                        index=None,
                        usecols=["mon", "day", "yr", "unit", "flo_out"],
                        filter_by={"unit": 1}
                        )
    res = reader.df
    res["Date"] = pd.to_datetime(pd.DataFrame({"year": res["yr"],
                                               "month": res["mon"],
                                               "day": res["day"]}))
    res.drop(columns=["mon", "day", "yr", "unit"], inplace=True)
    if delete_copy:
        shutil.rmtree(result, ignore_errors=True)
    os.chdir(cwd) #改回当前路径
    return res

# 定义SPOTPY类

In [3]:
class spot_swat():
    def __init__(self, TxtInOut_abspath, copy_path, start_print, end_print,
                 prior=sp.parameter.Uniform, obj_func=None,
                 show_output=False, delete_copy=True):
        self.reader = TxtinoutReader(TxtInOut_abspath)
        self.copy_path = copy_path
        self.start = start_print
        self.end = end_print
        self.obj_func = obj_func
        self.show_output = show_output
        self.delete_copy = delete_copy

        self.params = [prior("lat_ttime",0.51, 179.0, optguess = 39.70),
                       prior("can_max",0.01, 99.9, optguess = 89.84),
                       prior("esco",0.01,0.99, optguess = 0.018),
                       prior("epco",0.01, 0.99, optguess = 0.243),
                       prior("cn3_swf",0.01, 0.99, optguess = 0.613),
                       prior("perco",0.01, 0.99, optguess = 0.705),
                       prior("pet_co",0.71, 1.29, optguess = 0.79),
                       prior("latq_co",0.01, 0.99, optguess = 0.945),
                       prior("cn_a",30.0, 70.0, optguess = 49.2),
                       prior("cn_b",50.0, 80.0, optguess = 62.2),
                       prior("cn_c",70.0, 90.0, optguess = 89.3),
                       prior("cn_d",80.0, 95.0, optguess = 90.5),
                       prior("gw_flo",0.0, 2.0, optguess = 1.80),
                       prior("dep_bot",5.0, 50.0, optguess = 27.5),
                       prior("dep_wt",2.0, 20.0, optguess = 18.6),
                       prior("flo_dist",10.0, 200.0, optguess = 76.6),
                       prior("bf_max",0.11, 1.99, optguess =  1.49),
                       prior("alpha_bf",0.01, 0.99, optguess =  0.812),
                       prior("revap",0.021, 0.199, optguess = 0.119),
                       prior("rchg_dp",0.001, 0.100, optguess = 0.014),
                       prior("spec_yld",0.01, 0.49, optguess = 0.454),
                       prior("flo_min",0.1, 49.0, optguess = 23.9),
                       prior("revap_min",0.1, 49.0, optguess = 45.1),
                       prior("fall_tmp",-4.9, 4.9, optguess = -4.76),
                       prior("melt_tmp",-4.9, 4.9, optguess = 0.87),
                       prior("melt_max_min",0.1, 9.9, optguess = 9.28),
                       prior("melt_lag",0.01, 0.99, optguess = 0.253),
                       prior("snow_h2o",0.1, 499.9, optguess = 299.7),
                       prior("cov50",0.01, 0.99, optguess = 0.623),
                       prior("dp",0.1, 5999.9, optguess = 642.6),
                       prior("t_fc",0.1, 99.9, optguess = 9.45),
                       prior("lag",0.1, 99.9, optguess = 23.5),
                       prior("rad",3.1, 39.9, optguess = 21.0),
                       prior("dist",7601, 29999, optguess = 10673.9),
                       prior("drain",10.1, 50.9, optguess = 16.6),
                       prior("pump",0.1, 9.9, optguess = 8.80),
                       prior("lat_kast",0.02, 3.99, optguess = 3.82),
                       prior("lai_noevap",0.1, 9.9, optguess = 7.75),
                       prior("sw_init",0.01, 0.99, optguess = 0.021),
                       prior("surq_lag",1.1, 23.9, optguess = 13.1),
                       prior("msk_co1",0.1, 9.9, optguess = 2.49),
                       prior("msk_co2",0.1, 9.9, optguess = 5.99),
                       prior("msk_x",0.01, 0.29, optguess = 0.068),
                       prior("evap_adj",0.51, 0.99, optguess = 0.685),
                       prior("scoef",0.01, 0.99, optguess = 0.882),
                       prior("surq_exp",1.1, 2.9, optguess =  1.61),
                       ]
    def parameters(self):
        return sp.parameter.generate(self.params)

    def simulation(self, vector):
        par = np.array(vector)
        swat_params = {"hydrology.hyd":("name", [(None,"lat_ttime", 39.7),
                                                 (None,"can_max", 89.84),
                                                 (None,"esco", 0.018),
                                                 (None,"epco", 0.243),
                                                 (None,"cn3_swf", 0.613),
                                                 (None,"perco", 0.705),
                                                 (None,"pet_co", 0.79),
                                                 (None,"latq_co", 0.945),
                                                 ],
                                        ),
                       "cntable.lum":("description", [(None,"cn_a", 49.2),
                                                      (None,"cn_b", 62.2),
                                                      (None,"cn_c", 89.3),
                                                      (None,"cn_d", 90.5),
                                                      ],
                                      ),
                       "aquifer.aqu":("name", [(None, "gw_flo", 1.80),
                                               (None, "dep_bot", 27.5),
                                               (None, "dep_wt", 18.6),
                                               (None, "flo_dist", 76.6),
                                               (None, "bf_max", 1.49),
                                               (None, "alpha_bf", 0.812),
                                               (None, "revap", 0.119),
                                               (None, "rchg_dp", 0.014),
                                               (None, "spec_yld", 0.454),
                                               (None, "flo_min", 23.9),
                                               (None, "revap_min", 45.1),
                                              ],
                                      ),
                       "snow.sno":("name", [(None, "fall_tmp", -4.76),
                                            (None, "melt_tmp", 0.87),
                                            (None, "melt_max", 9.28),
                                            (None, "melt_min", 9.28),
                                            (None, "tmp_lag", 0.253),
                                            (None, "snow_h2o", 299.7),
                                            (None, "cov50", 0.623),
                                           ],
                                   ),
                       "tiledrain.str":("name", [(None, "dp", 642.6),
                                                 (None, "t_fc", 9.45),
                                                 (None, "lag", 23.5),
                                                 (None, "rad", 21.0),
                                                 (None, "dist", 10673.9),
                                                 (None, "drain", 16.6),
                                                 (None, "pump", 8.80),
                                                 (None, "lat_ksat", 3.82),
                                                ],
                                        ),
                       "parameters.bsn":("igen", [(None, "lai_noevap", 7.75),
                                                  (None, "sw_init", 0.021),
                                                  (None, "surq_lag", 13.1),
                                                  (None, "msk_co1", 2.49),
                                                  (None, "msk_co2", 5.99),
                                                  (None, "msk_x", 0.068),
                                                  (None, "evap_adj", 0.685),
                                                  (None, "scoef", 0.999),#0.882
                                                  (None, "surq_exp", 1.61),
                                                 ],
                                         ),
                       }
        sim = huron_swat(self.reader, swat_params, self.copy_path,
                         show_output=self.show_output, delete_copy=self.delete_copy)
        return sim["flo_out"]

    def evaluation(self):
        obs = pd.read_csv(os.path.join(cwd,'TimeSeries\\04199000.csv'))
        obs["Date"] = pd.to_datetime(obs["Date"])
        obs = obs.loc[((obs["Date"] >= self.start) & (obs["Date"] <= self.end)),"Q"]
        return obs
    def objectivefunction(self, simulation, evaluation):
        if not self.obj_func:
            like = sp.objectivefunctions.nashsutcliffe(evaluation, simulation)
        else:
            like = self.obj_func(evaluation, simulation)
        return like
    

In [4]:
# 源文件路径和复制文件路径
proj_path = os.path.join(cwd, "Best_TxtInOut")
copy_path = os.path.join(cwd, "Best_copy")

# 设置SWAT模拟时间范围
start_sim = "2017-01-01"
end_sim = "2020-12-31"

# 设置SWAT输出时间范围
start_print = "2018-01-01"
end_print = "2020-12-31"

# 输出选项
show_output = True
delete_copy = False

# 目标函数
obj_func = lambda evaluation, simulation: (
    -sp.objectivefunctions.nashsutcliffe(evaluation, simulation)
)

# 实例化及采样
spot_setup = spot_swat(proj_path, copy_path, start_print, end_print, obj_func=obj_func,
                       show_output=show_output, delete_copy=delete_copy)

spot_setup.reader.set_simulation_time(start_sim, end_sim)
spot_setup.reader.set_print_time(start_print, end_print)
spot_setup.reader.enable_object_in_print_prt("channel_sd", True, False, False, False)

sampler = sp.algorithms.mc(spot_setup,
                           dbname="Best", 
                           dbformat="csv",
                           )
sampler.sample(repetitions=2,
               # ngs=10,
               )

Initializing the  Monte Carlo (MC) sampler  with  2  repetitions
Starting the MC algorithm with 2 repetitions...
SWAT+
Revision 60.5.7
Soil & Water Assessment Tool
PC Version
Program reading . . . executing
Date of Sim   1/18/2024 Time  14:40:58
reading from precipitation file         Time  14:40:58
reading from temperature file           Time  14:40:58
reading from solar radiation file       Time  14:40:58
reading from relative humidity file     Time  14:40:58
reading from wind file                  Time  14:40:58
reading from wgn file                   Time  14:40:58
reading from wx station file            Time  14:40:58
Original Simulation        1   1  2017 Yr    1 of    4 Time  14:40:59
Original Simulation        1   2  2017 Yr    1 of    4 Time  14:40:59
Original Simulation        1   3  2017 Yr    1 of    4 Time  14:40:59
Original Simulation        1   4  2017 Yr    1 of    4 Time  14:40:59
Original Simulation        1   5  2017 Yr    1 of    4 Time  14:40:59
Original Simulation

Run number 0 has the lowest objectivefunction with: -0.1459


In [5]:
Best_results = sp.analyser.load_csv_results("Best")
bestindex, bestobjf = sp.analyser.get_minlikeindex(Best_results)
best_model_run = Best_results[bestindex]
fields = [word for word in best_model_run.dtype.names if word.startswith("sim")]
best_simulation = np.array(list(best_model_run[fields]))
obs = np.array(spot_setup.evaluation())

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18,18))
ax1.plot(Best_results['like1'])
ax1.set_xlabel("iteration")
ax1.set_ylabel("-NSE")

ax2.plot(obs, "r-", label="obs")
ax2.plot(best_simulation, "--", markersize = 2 ,label="best objf. = " + str(bestobjf))
ax2.set_xlabel("Number of obs. points")
ax2.set_ylabel("Q (m3/s)")
ax2.legend()

In [6]:
obs = np.array(spot_setup.evaluation())